In [ ]:
def residual_block(x, filters, kernel_size=3, stride=1, conv_shortcut=False):
    shortcut = x
    
    # 第一个卷积层
    y = Conv2D(filters, kernel_size=(kernel_size, kernel_size), strides=(stride, stride), padding='same')(x)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    
    # 第二个卷积层
    y = Conv2D(filters, kernel_size=(kernel_size, kernel_size), strides=(1, 1), padding='same')(y)
    y = BatchNormalization()(y)
    
    # 如果需要，使用卷积层调整捷径分支的形状
    if conv_shortcut or x.shape[-1] != filters:
        shortcut = Conv2D(filters, kernel_size=(1, 1), strides=(stride, stride), padding='same')(shortcut)
    
    # 添加捷径连接
    y = Add()([shortcut, y])
    y = Activation('relu')(y)
    
    return y


# 定义 ResNet 结构
def ResNet(input_shape=(30, 128, 1), num_classes=3):
    inputs = Input(shape=input_shape)
    
    # 第一个卷积层
    x = Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same')(x)
    
    # 堆叠残差块
    x = residual_block(x, filters=32, conv_shortcut=True)
    x = residual_block(x, filters=32)
    x = residual_block(x, filters=32)
    

    
    # 全局平均池化层
    x = GlobalAveragePooling2D()(x)
    
    # 全连接层
    x = Dense(num_classes, activation='softmax')(x)
    
    # 创建模型
    model = Model(inputs=inputs, outputs=x, name='ResNet')
    return model

# 创建 ResNet 模型
resnet = ResNet()

# 打印模型结构
resnet.summary()

# 编译模型
resnet.compile(optimizer='adam',  # 使用 Adam 优化器，设置学习率为 0.001
               loss='categorical_crossentropy',  # 多分类问题常用的交叉熵损失函数
               metrics=['accuracy'])  # 指定训练过程中监控的评估指标为准确率